In [1]:
#https://github.com/ThomasLamsonFr/AITextGenerator

In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoConfig, AutoTokenizer,GPT2Tokenizer, AdamW,get_linear_schedule_with_warmup
pd.set_option('display.max_colwidth', -1)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [3]:
pretrained_name_or_path = "gpt2-twitter-politico"

#### Define Cuda

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:',device)

Device: cuda


#### Cargamos el modelo pre-entrenado

In [5]:
model = AutoModelWithLMHead.from_pretrained(pretrained_name_or_path);
model.to(device);

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [6]:
print('Datos del Modelo:\n=================')
print(' - Tipo modelo Base:', model.config.model_type)
print(' - Arquitectura:', model.config.architectures)
print(' - Posiciones (largo máximo de secuencia):', model.config.n_positions)
print(' - Tamaño dimensional interno:', model.config.n_embd)
print(' - Cabezales de Atención:', model.config.n_head)
print(' - Capas :', model.config.n_layer)
print(' - Tamaño de Vocabulario :', model.config.vocab_size)
print(' - Function de Activación :', model.config.activation_function)
#GELU: https://medium.com/@shoray.goel/gelu-gaussian-error-linear-unit-4ec59fb2e47c

Datos del Modelo:
 - Tipo modelo Base: gpt2
 - Arquitectura: ['GPT2LMHeadModel']
 - Posiciones (largo máximo de secuencia): 1024
 - Tamaño dimensional interno: 768
 - Cabezales de Atención: 12
 - Capas : 12
 - Tamaño de Vocabulario : 50264
 - Function de Activación : gelu_new


#### Cargamos nuestro tokenizador español

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_name_or_path);

In [8]:
print('Datos del Tokenizador:\n======================')
print(' - Tamaño del vocabulario:', tokenizer.vocab_size)
print(' - Token Inicio de Secuencia (token => id): {} => {}'.format(tokenizer.bos_token, tokenizer.bos_token_id))
print(' - Token Fin de Secuencia (token => id): {} => {}'.format(tokenizer.eos_token, tokenizer.eos_token_id))
print(' - Token de relleno (token => id): {} => {}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
print(' - Token fuera de vocabulario (token => id): {} => {}'.format(tokenizer.unk_token, tokenizer.unk_token))
print(' - Token mascara (token => id): {} => {}'.format(tokenizer.mask_token, tokenizer.mask_token))
print(' - Largo máximo: {}'.format(tokenizer.max_len))
# https://huggingface.co/transformers/main_classes/tokenizer.html

Using mask_token, but it is not set yet.
Using mask_token, but it is not set yet.


Datos del Tokenizador:
 - Tamaño del vocabulario: 50257
 - Token Inicio de Secuencia (token => id): [TWEET] => 50257
 - Token Fin de Secuencia (token => id): <|endoftext|> => 50256
 - Token de relleno (token => id): <|endoftext|> => 50256
 - Token fuera de vocabulario (token => id): <|endoftext|> => <|endoftext|>
 - Token mascara (token => id): None => None
 - Largo máximo: 1024


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


#### Un ejemplo del tokenizador en funcionamiento
Ubica las palabras en su representación vectorial ya aprendida, una palabra nueva la descompone utilizando tokens ya conocidos

### Nuestras Pruebas deben tener esta estructura

Cada tweet lo vamos a estructurar de la siguiente forma para entregarle a nuestro modelo cada ejemplo así:

|Coalición|Partido|Sentimiento|Entidades|Frases Clave| tweet |
|---      |---    |---        |---      |---           | ---| 
|[COALICION] chile vamos|[PARTIDO] udi |[SENTIMIENTO] positivo |[ENTIDADES] carabineros  |[FRASES] cuentan con el apoyo| [TWEET]

In [9]:
pruebas = [
    {"COALICION":"Chile Vamos","PARTIDO":"UDI","SENTIMIENTO":"NEUTRAL","ENTIDADES":"sebastianpinera","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Chile Vamos","PARTIDO":"RN","SENTIMIENTO":"NEGATIVE","ENTIDADES":"","HASHTAGS":"araucania","FRASES":"","TWEET":""},
    {"COALICION":"Frente Amplio","PARTIDO":"PH","SENTIMIENTO":"NEGATIVE","ENTIDADES":"gobierno","HASHTAGS":"araucania","FRASES":"ddhh","TWEET":""},
    {"COALICION":"Nueva Mayoria","PARTIDO":"","SENTIMIENTO":"POSITIVE","ENTIDADES":"plebiscito","HASHTAGS":"apruebo","FRASES":"nosotros decidimos","TWEET":""},
    {"COALICION":"Frente Amplio","PARTIDO":"","SENTIMIENTO":"POSITIVE","ENTIDADES":"cuba","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Independiente","PARTIDO":"","SENTIMIENTO":"POSITIVE","ENTIDADES":"cuba","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"","PARTIDO":"RD","SENTIMIENTO":"NEGATIVE","ENTIDADES":"Trump","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Chile Vamos","PARTIDO":"","SENTIMIENTO":"POSITIVE","ENTIDADES":"covid-19","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Chile Vamos","PARTIDO":"","SENTIMIENTO":"NEGATIVE","ENTIDADES":"covid-19","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Frente Amplio","PARTIDO":"","SENTIMIENTO":"POSITIVE","ENTIDADES":"covid-19","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Frente Amplio","PARTIDO":"","SENTIMIENTO":"NEGATIVE","ENTIDADES":"covid-19","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Nueva Mayoría","PARTIDO":"","SENTIMIENTO":"POSITIVE","ENTIDADES":"covid-19","HASHTAGS":"","FRASES":"","TWEET":""},
    {"COALICION":"Nueva Mayoría","PARTIDO":"","SENTIMIENTO":"NEGATIVE","ENTIDADES":"covid-19","HASHTAGS":"","FRASES":"","TWEET":""}
]
pruebas_df = pd.DataFrame.from_records(pruebas)
pruebas_df

,COALICION,PARTIDO,SENTIMIENTO,ENTIDADES,HASHTAGS,FRASES,TWEET
0,Chile Vamos,UDI,NEUTRAL,sebastianpinera,,,
1,Chile Vamos,RN,NEGATIVE,,araucania,,
2,Frente Amplio,PH,NEGATIVE,gobierno,araucania,ddhh,
3,Nueva Mayoria,,POSITIVE,plebiscito,apruebo,nosotros decidimos,
4,Frente Amplio,,POSITIVE,cuba,,,
5,Independiente,,POSITIVE,cuba,,,
6,,RD,NEGATIVE,Trump,,,
7,Chile Vamos,,POSITIVE,covid-19,,,
8,Chile Vamos,,NEGATIVE,covid-19,,,
9,Frente Amplio,,POSITIVE,covid-19,,,


In [10]:
import os
from src.torch_loader import  DatasetFromPandas, VectorizeMode, VectorizeParagraph
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

from tqdm.notebook import tqdm, trange

import random
from torch.utils.tensorboard import SummaryWriter
from src.flexible_models.flexible_GPT2 import FlexibleGPT2


ModuleNotFoundError: No module named 'src.flexible_models'

In [117]:
GPT2_BLOCK_SIZE = model.config.n_positions

vectorizer = VectorizeParagraph( tokenizer=tokenizer, mode=VectorizeMode.EVAL)

dataset = DatasetFromPandas(pruebas_df, vectorizer)

# Generate Texts

In [25]:
def pad_left_side(sequences, padding_value):
    """
    Modification of torch.nn.utils.rnn.pad_sequence so that we pad left side and not right side
    :param sequences : list of tensors
    :param padding_value : tokenizer.pad_token_id
    :return tensor of shape (len(sequences), max_length of sequence in sequences)
            the tensor are padded on the left side using pad_token_id from GPT2 tokenizer
    """
    max_len = max([s.size(0) for s in sequences])
    out_dims = (len(sequences), max_len)
    out_tensor = sequences[0].data.new(*out_dims).fill_(padding_value)
    for i, tensor in enumerate(sequences):
        length = tensor.size(0)
        out_tensor[i, max_len - length:] = tensor
    return out_tensor

In [88]:
def generate_texts(dataset, GPT2_model, verbose=1):

    def custom_collate(data_samples):
        collate_input_ids = pad_left_side([single_sample[0] for single_sample in data_samples],
                                          padding_value=GPT2_model.tokenizer.pad_token_id)
        original_samples = [single_sample[1] for single_sample in data_samples]
        return collate_input_ids, original_samples

    dataloader = DataLoader(dataset=dataset, batch_size=10, collate_fn=custom_collate)

    generations, originals = [], []
    
    n_gen=1
    for input_ids, samples in dataloader:
        print ('generando...', n_gen)
        n_gen +=1
        generations += GPT2_model(input_ids)
        originals += samples
        
    return generations

In [111]:
bad_words_ids =[ 
    tokenizer.encode('…', add_prefix_space=True),
    tokenizer.encode('&', add_prefix_space=True)
]



DEFAULT_DECODING_STRATEGY = {
    'do_sample': True,
    'min_length': 50,
    'max_length': 150,
    'top_p': 0.9,
    'temperature':0.9,
    'num_beams': 10,
    'bad_words_ids': bad_words_ids,
    'early_stop':True,
    'no_repeat_ngram_size': 2,
    'length_penalty':2
}

gpt_2 = FlexibleGPT2(model=model, tokenizer=tokenizer, decoding_strategy=DEFAULT_DECODING_STRATEGY)

In [118]:
%%time
gen_tweets = generate_texts(dataset, gpt_2)

Generating texts...generando... 1
generando... 2
CPU times: user 14.4 s, sys: 2.17 s, total: 16.5 s
Wall time: 16.5 s


In [119]:
cleaned_tweets = [ (t
   .replace('<unk>','')
   .replace('|>','')
   .replace('|endoftext','')
   .replace('|endoftex','')
   .replace('|endofte','')
   .replace('|endoft','')
   .replace('|endof','')
   .replace('|endo','')
   .replace('|end','')
   .replace('<|','')
   .replace('|','')
   .replace('|','')
   .replace('<','').strip()
  ) for t in gen_tweets ]

tweets_df = pd.DataFrame.from_records([{'TWEET': t} for t in cleaned_tweets])
result = pd.concat([pruebas_df, tweets_df],axis=1)
result

,COALICION,PARTIDO,SENTIMIENTO,ENTIDADES,HASHTAGS,FRASES,TWEET,TWEET
0,Chile Vamos,UDI,NEUTRAL,sebastianpinera,,,,"Pdte. @_alecandia se reúne con alcaldes y seremis de todo el país para coordinar y coordinar acciones en beneficio de la comunidad > ➡️Para todos los sectores 🤦‍♂para los más vulnerables Revisa el detalle de las medidas anunciadas por el ppto.@GobiernodeChile⁩, en el siguiente link ⬇⤵✅¡Comparte y difunde con los alcaldes de todas las regiones del país! ☎⁦"
1,Chile Vamos,RN,NEGATIVE,,araucania,,,"Los niños no pueden esperar, el tiempo no puede significar impunidad y el abuso sexual de menores no tiene perdón, por eso es necesario que se apruebe el proyecto de ley que declara de interés nacional la explotación y comercialización de niños, niñas y adolescentes 👨‍⚕️ ✅Recuperar el consumo de drogas y drogas en niños y niñas. deporte @DiputadosRNchile⁩ ⁦@PDI_CHILE⁦ y ➡�"
2,Frente Amplio,PH,NEGATIVE,gobierno,araucania,ddhh,,El @gobierno es el que tiene que dar la cara y no dar cuenta de lo que está pasando en el país 🤦‍♂️ > de la violencia y el vandalismo. Espero que el gobierno escuche a la gente y pida a las personas que se manifiestan pacíficamente en las calles y en contra de los DDHH de las dictaduras de nuestro país. ⁦@elfrente_amplio⁩ [1] NO]!! NO+1⁦#ara
3,Nueva Mayoria,,POSITIVE,plebiscito,apruebo,nosotros decidimos,,"Porque queremos una nueva constitución, con tod@s y respetos. 💜✊Súmate a la campaña del terror contra el abuso y la impunidad.➡️ Ingresa a y participa. en la consulta individual de @elfrente_amplio.cl o llama a votar en o en el plebiscito por una Nueva Constitución. ☝� &ampliado.gob@congreso.c Contáctanos.&gt;&lt;"
4,Frente Amplio,,POSITIVE,cuba,,,,Seguimos trabajando para mejorar la calidad de vida de las y los trabajadores 💪 deporte ➡️ Si eres trabajador/a a honorarios y cotizaste@s en el mundo del trabajo.⁦@PCdeChile⁩ @elfrente_amplio [1] NO+AFPATRIO]♻3% de cotizacionessenado.cl#paridadteisosenador@chahuan@conce #participación de trabajadores/es del
5,Independiente,,POSITIVE,cuba,,,,"En el día de la mujer 🙋‍♀️, quiero saludar a todas las mujeres de nuestra comuna, en especial a mi querida juventud conconina. deporte Un abrazo fraterno para todas y todos.•Gracias por todo lo que hacen por hacer de mi vida un país más justo y digno para tod@s, un mundo mejor para nosotras, muchas felicidades a nuestras queridas mujeres y hombres que dan vida a nuestro país @Muni_LaCisteria_CL"
6,,RD,NEGATIVE,Trump,,,,"@_jwe No me hago cargo 😅 FFA y de Orden EL PAÍS Retina @elfrente_amplio @la_convergencia [barroco ] _sebagarro DDH DDHH y no me he dado cuenta que he sido parte de la historia de las violaciones a los derechos humanos durante el estado de emergencia, por lo tanto es inaceptable lo que ha sido hasta el momento!!➡️NO"
7,Chile Vamos,,POSITIVE,covid-19,,,,"Tenemos que cuidarnos entre todos y todas, es por eso que hemos puesto a disposición de la ciudadanía la plataforma para que podamos apoyar a las personas en situación de calle ante el contagio del Covid_19. > 😷➡️Para saber si eres víctima o testigo de violencia contra la mujer y no sabes qué hacer en casa, recuerda que puedes llamar al Fono Orientación ☎141455, disponible 24/7 para recibir orientación y ayuda de apoyo y orientación.¡Difunde con"
8,Chile Vamos,,NEGATIVE,covid-19,,,,📌 En vivo por @biobio conversando sobre la contingencia por covid_19 y respondiendo las dudas y consultas de los vecinos. > ➡️ Balance de HOY con los casos CONFIRMADOS de cada comuna y contáctanos de contagios.&gt;FUEREMOSFono Fono Orientación ☎141455 �⏰Y en vivo y en directo a través de Fonasa.go fmg Contacto
9,Frente Amplio,,POSITIVE,covid-19,,,,"Aprovechemos de proteger a los trabajadores y trabajadoras de nuestro país frente a la crisis del Covid_19 😷 FF.AA. ➡️ Balance de HOYas con los casos CONFIRMADOS de la @Camara_cl Revisa el detalle completo de las medidas adoptadas por el gobierno para resguardar la salud de los trabajador@s, en el marco de esta crisis sanitaria y l

In [113]:
cleaned_tweets = [ (t
   .replace('<unk>','')
   .replace('|>','')
   .replace('|endoftext','')
   .replace('|endoftex','')
   .replace('|endofte','')
   .replace('|endoft','')
   .replace('|endof','')
   .replace('|endo','')
   .replace('|end','')
   .replace('<|','')
   .replace('|','')
   .replace('|','')
   .replace('<','').strip()
  ) for t in gen_tweets ]

tweets_df = pd.DataFrame.from_records([{'TWEET': t} for t in cleaned_tweets])
result = pd.concat([pruebas_df, tweets_df],axis=1)
result

,COALICION,PARTIDO,SENTIMIENTO,ENTIDADES,HASHTAGS,FRASES,TWEET,TWEET
0,Chile Vamos,UDI,NEUTRAL,sebastianpinera,,,,"Pdte. @_alecandia se reúne con alcaldes y seremis de todo el país para coordinar acciones que contribuyan a mejorar la calidad de vida de las personas > 🤦‍♂️✅Seguimos trabajando juntos por el bien de nuestro país, con el objetivo de ir en ayuda de quienes más lo necesitan ➕infraestructura y recursos para ell@s más vulnerables❗⁦@GobiernodeChile⁩ ⬇⭐⚖ �"
1,Chile Vamos,RN,NEGATIVE,,araucania,,,"En plena pandemia, los niños no pueden esperar, el tiempo no puede significar impunidad y el abuso sexual de menores no tiene perdón, por eso es necesario que se tomen las medidas de prevención y se respeten los derechos de niños, niñas y adolescentes 👨‍⚕️✅Se debe respetar el estado de derecho y la seguridad de todos los menores de edad, y no arriesgarlos a la salud de niñ@s y jóvenes. > @DiputadosRNchile⁩ ✔"
2,Frente Amplio,PH,NEGATIVE,gobierno,araucania,ddhh,,El @gobierno es el que tiene que dar la cara al gobierno 🤦‍♂️ > de la @la_convergencia⁩ ⁦@elfrente_amplio [1] NO+0⃣⁦�DONOVACIÓN NACIONAL Todo ell@s que se movilizan para frenar la violencia contra la mujer y el gobierno que busca desprestigiar el movimiento social EL PAÍS FAMILIARLOS #ara


In [106]:
cleaned_tweets[2]

'El @gobierno no puede pretender que al gobierno le vaya mal al país 🤦\u200d♂️      \u2066@elfrente_amplio\u2069 &gt; [barricadaste]  EL PAÍS FFAA  Radio Universal  El Desconcierto  Para tod@s lxs el gobierno no le hace bien a la gente en las calles y no a los DDHH de las victimas de la violencia y el vandalismo puro  Araucanía Noticias  24horas'